In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import json

In [ ]:

school_districts_gpd = gpd.read_file("../data/sd-geo/a0000000c.gdbtable") 
school_districts_gpd["geometry"] = school_districts_gpd["geometry"].simplify(tolerance=0.01,preserve_topology=True) 
# NOTE: this tolerance is in units of degrees of lat/lon, not any distance unit
school_districts_gpd.index = school_districts_gpd["GEOID"].tolist()


keep_cols = ["GEOID","NAME","geometry"]
with open("geojson-school-districts.json","w") as fp:
    print(school_districts_gpd[keep_cols].to_json(),file=fp)


with open("geojson-school-districts-mini.json","w") as fp:
    print(school_districts_gpd[keep_cols].sample(500).to_json(),file=fp)

In [ ]:
with open("../geojson-counties-fips.json") as fp:
    counties = json.load(fp)

with open("../geojson-school-districts.json") as fp:
    school_districts = json.load(fp)


In [ ]:

df = pd.read_csv("../cty-sd-latest-indicators.csv")
df["state"] = df["geo_name"].apply(lambda name: name.split(", ")[-1])

df_cty = df[df.geo_type == 'CTY'].copy(deep=True)
df_sd  = df[df.geo_type == 'SD'].copy(deep=True)

df_cty["fips"] =  df_cty["geo_id"].apply(lambda id: id[-5:])
df_sd["district"] = df_sd["geo_id"].apply(lambda id: id[-7:])

In [ ]:


fig = px.choropleth(
    df_cty[["fips","pct_limited_english_households","geo_name"]], geojson=counties, locations='fips', color='pct_limited_english_households',
    color_continuous_scale="Viridis",
    range_color=(-1, 8),
    scope="usa",
    labels={'pct_limited_english_households':'households with limited english','geo_name':'county name'}
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:


states = ["New York","Vermont","Utah","Colorado"]
fig_sd = px.choropleth(
    df_sd[df_sd.state.isin(states)][["district","pct_limited_english_households","geo_name"]], geojson=school_districts, locations='district', color='pct_limited_english_households',
    color_continuous_scale="Viridis",
    range_color=(-1, 8),
    scope="usa",
    labels={'pct_limited_english_households':'households with limited english','geo_name':'county name'}
)

fig_sd.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig_sd.show()

# https://gis.stackexchange.com/questions/325766/geopandas-simplify-results-in-gaps-between-polygons
# can fix the gaps this way above. although to be honest I don't  get any additional information
# from added granularity of the district plot. I wil be sticking with county-level for future plotting